# Reporte 06
- Luis Jiménez
- José Melchor
- Lenin Quezada
- Édgar Ramos
---

### Actividad de la Sesión

1. **Discusión en clase**: Analizaremos ejemplos de pruebas A/B reales y discutiremos cómo aplicar esta metodología en diferentes áreas de negocio, como marketing, cambios en ambiente productivo, diseño de productos y fijación de precios.
2. **Trabajo en equipo**: Diseñen una prueba A/B para evaluar una hipótesis de cambio de precio.

#### Trabajo de equipo
CompuTech, una empresa de servicios de consulta de AI a través de API, está evaluando cambiar el precio de uno de sus servicios más populares. Tienen como objetivo incrementar 20 USD el ingreso promedio por usuario.


El servicio tiene actualmente un precio de 1 USD por consulta, y la empresa está considerando aumentar el precio a 1.25 USD. Antes de implementar este cambio, desean evaluar cómo este aumento afectará su KPI ventas por usuario. Se llevará a cabo una prueba A/B para respaldar esta decisión.

##### Objetivo del Caso
El objetivo de la prueba es determinar si el cambio de precio tiene un impacto significativo en los siguientes KPIs:

Ingreso promedio por usuario: Ingreso promedio por usuario. Desean que por lo menos aumente un 20 USD.

Tasa de Conversión: Proporción de usuarios que completan la suscripción.


##### Hipótesis
- Hipótesis Nula ($H_0$): El aumento de precio a 1.25 USD no tiene un impacto significativo en la tasa de conversión ni en el ingreso promedio por usuario.
- Hipótesis Alternativa ($H_1$): El aumento de precio a 1.25 USD tiene un impacto significativo en la tasa de conversión y el ingreso promedio por usuario.


In [1]:
import pandas as pd
import statsmodels.api as sm
import scipy.stats as stats
import numpy as np

**Prueba de hipótesis de tasa conversión:**
- $H_0: \Delta TC \leq 0$
- $H_i: \Delta TC > 0$

**Prueba de hipótesis de tasa ARPU:**
- $H_0: \Delta TC \leq 20$
- $H_i: \Delta TC > 20$

### Importar Datos

In [2]:
conversion_A = pd.read_csv('group_A_conversion.csv')
conversion_B = pd.read_csv('group_B_conversion.csv')
revenue_A = pd.read_csv('group_A_revenue.csv')
revenue_B = pd.read_csv('group_B_revenue.csv')
historical_arpu = pd.read_csv('historical_arpu.csv')

In [3]:
len(revenue_A), len(revenue_B), len(conversion_A), len(conversion_B)

(800, 200, 1000, 1000)

In [4]:
conversion_A['Converted'].mean(), conversion_B['Converted'].mean()

(0.043, 0.04)

### Tamaños de Muestra

In [14]:
n_tc = (2*((1.96+0.84)**2)*0.043*(1-0.043))/(0.043-0.04)**2
print('n óptimo para tasa de conversión:', n_tc)

n óptimo para tasa de conversión: 71694.18666666684


In [15]:
var_arpu = (historical_arpu['ARPU'].std())**2
n_arpu = (2*((1.96+0.84)**2)*var_arpu)/(20)**2
print('n óptimo para ARPU:', n_arpu)

n óptimo para ARPU: 8.457179188620609


### Pruebas Estadísticas

In [16]:
# Experimento controlado, distinto número de observaciones
arpu_A = revenue_A['ARPU']
arpu_B = revenue_B['ARPU']

# Objectivo: tener diferencia de 20 dolares en el ARPU
objective_difference = 20

# Promedio y desviación estandar por grupo
mean_a = np.mean(arpu_A)
mean_b = np.mean(arpu_B)

std_a = np.std(arpu_A, ddof=1)
std_b = np.std(arpu_B, ddof=1)

n_a = len(arpu_A)
n_b = len(arpu_B)

pooled_std = np.sqrt(((std_a**2 / n_a) + (std_b**2 / n_b)))
t_stat = (mean_b - mean_a - objective_difference) / pooled_std
p_value = 1 - stats.t.cdf(t_stat, df=min(n_a, n_b) - 1)

# Output the results
print(f"Mean LTV of Group A: {mean_a:.2f}")
print(f"Mean LTV of Group B: {mean_b:.2f}")
print(f"Objective Difference: {objective_difference}")
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

Mean LTV of Group A: 100.06
Mean LTV of Group B: 120.75
Objective Difference: 20
T-statistic: 0.4893
P-value: 0.3126


In [17]:
# Experimento controlado, distinto número de observaciones
arpu_A = revenue_A['ARPU']
arpu_B = revenue_B['ARPU']

# Objectivo: tener diferencia de 18.35 dolares en el ARPU
objective_difference = 18.35

# Promedio y desviación estandar por grupo
mean_a = np.mean(arpu_A)
mean_b = np.mean(arpu_B)  
std_a = np.std(arpu_A, ddof=1)
std_b = np.std(arpu_B, ddof=1)

n_a = len(arpu_A)
n_b = len(arpu_B)

pooled_std = np.sqrt(((std_a**2 / n_a) + (std_b**2 / n_b)))
t_stat = (mean_b - mean_a - objective_difference) / pooled_std
p_value = 1 - stats.t.cdf(t_stat, df=min(n_a, n_b) - 1)

# Output the results
print(f"Mean LTV of Group A: {mean_a:.2f}")
print(f"Mean LTV of Group B: {mean_b:.2f}")
print(f"Objective Difference: {objective_difference}")
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

Mean LTV of Group A: 100.06
Mean LTV of Group B: 120.75
Objective Difference: 18.35
T-statistic: 1.6539
P-value: 0.0499


In [18]:
conversions_a = conversion_A['Converted'].sum()
total_a = len(conversion_A['Converted'])
conversions_b = conversion_B['Converted'].sum()
total_b = len(conversion_B['Converted'])

# Calcular conversion rate
rate_a = conversions_a / total_a
rate_b = conversions_b / total_b

# Hacer un z-test de proporciones
count = np.array([conversions_a, conversions_b])
nobs = np.array([total_a, total_b])
z_stat, p_value = sm.stats.proportions_ztest(count, nobs)

print(f"Z-statistic: {z_stat}, P-value: {p_value}")

Z-statistic: 0.33634590917760526, P-value: 0.7366100338880623


### Conclusiones

- No se puede aceptar que el incrementar 1.25 USD el precio se obtenga una diferencia de 20 USD en el ARPU dado que el p-value es de 0. 3126.
- Sin embargo, se puede aceptar que el incrementar 1.25 USD obtenga una diferencia de 18.35 USD en el ARPU dado que el p-value es de 0.0499.
- Tambien, no podemos aceptar que la tasa de conversión se mantendrá igual a 0 dado que el p-value es de 0.7366. Aunque hay un 73% de probabilidades de que suceda, tambien hay altas posibilidades de que incremente.